In [1]:
import pandas as pd
import geopandas as gpd
import time
import os
start_time = time.time()

In [3]:
print(pd.__version__)
print(gpd.__version__)

1.1.1
0.8.1


In [4]:
os.chdir('..')

In [5]:
## Load encoded restaurant license data
dc_food = pd.read_csv("data\\interim\\DC_FoodEstablish_Since2005.csv", low_memory = False)

## Load neighborhood areas
dc_neighborhood_area = pd.read_csv("data\\interim\\neigbh_area.csv", low_memory = False)

## Load geojson as geopandas dataframe
dc_zil_gdf = gpd.read_file("data\\external\\zillow_nb_dc.geojson")    

dc_zil_gdf.head()

city                      name regionid                county state  \
0  Washington       Catholic University   273159  District of Columbia    DC   
1  Washington            McLean Gardens   121759  District of Columbia    DC   
2  Washington             Benning Ridge   403493  District of Columbia    DC   
3  Washington  Southwest Federal Center   403489  District of Columbia    DC   
4  Washington          Civic Betterment   403495  District of Columbia    DC   

                                            geometry  
0  POLYGON ((-77.00433 38.94064, -77.00423 38.940...  
1  POLYGON ((-77.07520 38.93977, -77.07475 38.938...  
2  POLYGON ((-76.93493 38.87296, -76.93501 38.872...  
3  POLYGON ((-77.01609 38.88757, -77.01552 38.887...  
4  POLYGON ((-76.93053 38.88062, -76.93040 38.880...

# 2.3a Calculate Summary Metrics

In [6]:
dc_rest = dc_food[(dc_food.LICENSE_END_DATE >= '2018-01-01') 
                  & (dc_food.LICENSECATEGORY.isin(['Restaurant','Delicatessen']))
                 ].copy()

In [7]:
## Returns 1 if license is active during time period of interest and specified category
def active(col1, col2, col3, col4, license, date):
    if (col1 == 'Ready to Renew' or col1 == 'Active') and col2 >= date and col3 <= date and col4 == license:
        return 1
    else:
        return 0  
    
## Returns 1 if license is not active, license is within time period of interest and license is specified category

def closed(col1, col2, col3, license, end, start):
    if col1 != 'Ready to Renew' and col1 != 'Active' and col2 <= end and col2 >= start and col3 == license:
        return 1
    else:
        return 0   

In [8]:
## Apply functions to create columns for restaurants and delis
dc_rest['active_rest'] = dc_rest.apply(lambda x: active(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSE_START_DATE'],x['LICENSECATEGORY'],'Restaurant', '2019-09-01'), axis = 1)
dc_rest['close_18_rest'] = dc_rest.apply(lambda x: closed(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSECATEGORY'],'Restaurant', '2018-12-31', '2017-12-31'), axis = 1)
dc_rest['close_19_rest'] = dc_rest.apply(lambda x: closed(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSECATEGORY'],'Restaurant', '2019-12-31', '2018-12-31'), axis = 1)

dc_rest['active_deli'] = dc_rest.apply(lambda x: active(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSE_START_DATE'],x['LICENSECATEGORY'],'Delicatessen', '2019-09-01'), axis = 1)
dc_rest['close_18_deli'] = dc_rest.apply(lambda x: closed(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSECATEGORY'],'Delicatessen', '2018-12-31', '2017-12-31'), axis = 1)
dc_rest['close_19_deli'] = dc_rest.apply(lambda x: closed(x['LICENSESTATUS'],x['LICENSE_END_DATE'],x['LICENSECATEGORY'],'Delicatessen', '2019-12-31', '2018-12-31'), axis = 1)


In [9]:
## Roll-up the data to the neighborhood level
dc_rest_grp = dc_rest.groupby("neighborhood")['active_rest','close_18_rest','close_19_rest','active_deli','close_18_deli','close_19_deli'].sum()

## Create a couple of additional counts
dc_rest_grp['active_tot'] = dc_rest_grp['active_rest'] + dc_rest_grp['active_deli']
dc_rest_grp['close_18_tot'] = dc_rest_grp['close_18_rest'] + dc_rest_grp['close_18_deli']
dc_rest_grp['close_19_tot'] = dc_rest_grp['close_19_rest'] + dc_rest_grp['close_19_deli']
dc_rest_grp['pct_closed_rest'] = 100*round((dc_rest_grp['close_19_rest'] + dc_rest_grp['close_18_rest'])/(dc_rest_grp['close_19_rest'] + dc_rest_grp['close_18_rest']+dc_rest_grp['active_rest']),3)
dc_rest_grp['pct_closed_deli'] = 100*round((dc_rest_grp['close_19_deli'] + dc_rest_grp['close_18_deli'])/(dc_rest_grp['close_19_deli'] + dc_rest_grp['close_18_deli']+dc_rest_grp['active_deli']),3)
dc_rest_grp['pct_closed_tot'] = 100*round((dc_rest_grp['close_19_tot'] + dc_rest_grp['close_18_tot'])/(dc_rest_grp['close_19_tot'] + dc_rest_grp['close_18_tot']+dc_rest_grp['active_tot']),3)

## Replace NAs with 0
dc_rest_grp.fillna(0, inplace = True)

dc_rest_grp.head()

<ipython-input-9-60be14648faf>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dc_rest_grp = dc_rest.groupby("neighborhood")['active_rest','close_18_rest','close_19_rest','active_deli','close_18_deli','close_19_deli'].sum()


active_rest  close_18_rest  \
neighborhood                                                                   
Adams Morgan                                               83             16   
American University Park                                   26              3   
Anacostia                                                   1              0   
Anacostia Naval Station - Boiling Air Force Base            1              0   
Arboretum                                                   0              0   

                                                  close_19_rest  active_deli  \
neighborhood                                                                   
Adams Morgan                                                  9           19   
American University Park                                      1            5   
Anacostia                                                     0            6   
Anacostia Naval Station - Boiling Air Force Base              0            1   
Arboretum                                                     0            1   

                                                  close_18_deli  \
neighborhood                                                      
Adams Morgan                                                  3   
American University Park                                      0   
Anacostia                                                     1   
Anacostia Naval Station - Boiling Air Force Base              0   
Arboretum                                                     0   

                                                  close_19_deli  active_tot  \
neighborhood                                                                  
Adams Morgan                                                  3         102   
American University Park                                      0          31   
Anacostia                                                     3           7   
Anacostia Naval Station - Boiling Air Force Base              0           2   
Arboretum                                                     2           1   

                                                  close_18_tot  close_19_tot  \
neighborhood                                                                   
Adams Morgan                                                19            12   
American University Park                                     3             1   
Anacostia                                                    1             3   
Anacostia Naval Station - Boiling Air Force Base             0             0   
Arboretum                                                    0             2   

                                                  pct_closed_rest  \
neighborhood                                                        
Adams Morgan                                                 23.1   
American University Park                                     13.3   
Anacostia                                                     0.0   
Anacostia Naval Station - Boiling Air Force Base              0.0   
Arboretum                                                     0.0   

                                                  pct_closed_deli  \
neighborhood                                                        
Adams Morgan                                                 24.0   
American University Park                                      0.0   
Anacostia                                                    40.0   
Anacostia Naval Station - Boiling Air Force Base              0.0   
Arboretum                                                    66.7   

                                                  pct_closed_tot  
neighborhood                                                      
Adams Morgan                                                23.3  
American University Park                                    11.4  
Anacostia                                                   36.4  
Anacostia Naval Station - Boiling Air Fo

In [10]:
### If number of food establishments is less than 5, 0 out the percent closed metric
def min_limit(col1, col2):
    if col1 <= 5:
        return 0
    else:
        return col2
    
dc_rest_grp['pct_closed_tot'] = dc_rest_grp.apply(lambda x: min_limit(x['active_tot'],x['pct_closed_tot']), axis = 1)
dc_rest_grp['pct_closed_rest'] = dc_rest_grp.apply(lambda x: min_limit(x['active_rest'],x['pct_closed_rest']), axis = 1)
dc_rest_grp['pct_closed_deli'] = dc_rest_grp.apply(lambda x: min_limit(x['active_deli'],x['pct_closed_deli']), axis = 1)

dc_rest_grp.head()

active_rest  close_18_rest  \
neighborhood                                                                   
Adams Morgan                                               83             16   
American University Park                                   26              3   
Anacostia                                                   1              0   
Anacostia Naval Station - Boiling Air Force Base            1              0   
Arboretum                                                   0              0   

                                                  close_19_rest  active_deli  \
neighborhood                                                                   
Adams Morgan                                                  9           19   
American University Park                                      1            5   
Anacostia                                                     0            6   
Anacostia Naval Station - Boiling Air Force Base              0            1   
Arboretum                                                     0            1   

                                                  close_18_deli  \
neighborhood                                                      
Adams Morgan                                                  3   
American University Park                                      0   
Anacostia                                                     1   
Anacostia Naval Station - Boiling Air Force Base              0   
Arboretum                                                     0   

                                                  close_19_deli  active_tot  \
neighborhood                                                                  
Adams Morgan                                                  3         102   
American University Park                                      0          31   
Anacostia                                                     3           7   
Anacostia Naval Station - Boiling Air Force Base              0           2   
Arboretum                                                     2           1   

                                                  close_18_tot  close_19_tot  \
neighborhood                                                                   
Adams Morgan                                                19            12   
American University Park                                     3             1   
Anacostia                                                    1             3   
Anacostia Naval Station - Boiling Air Force Base             0             0   
Arboretum                                                    0             2   

                                                  pct_closed_rest  \
neighborhood                                                        
Adams Morgan                                                 23.1   
American University Park                                     13.3   
Anacostia                                                     0.0   
Anacostia Naval Station - Boiling Air Force Base              0.0   
Arboretum                                                     0.0   

                                                  pct_closed_deli  \
neighborhood                                                        
Adams Morgan                                                 24.0   
American University Park                                      0.0   
Anacostia                                                    40.0   
Anacostia Naval Station - Boiling Air Force Base              0.0   
Arboretum                                                     0.0   

                                                  pct_closed_tot  
neighborhood                                                      
Adams Morgan                                                23.3  
American University Park                                    11.4  
Anacostia                                                   36.4  
Anacostia Naval Station - Boiling Air Fo

# 2.3b Create GeoPandas Dataframe

In [11]:
### merge the variables of interest into the Geodataframe
dc_gdf = dc_zil_gdf.merge(dc_rest_grp, left_on='name', right_on='neighborhood', how='left')

### merge the area of each neighborhood
dc_gdf = dc_gdf.merge(dc_neighborhood_area)
dc_gdf.head()

city                      name regionid                county state  \
0  Washington       Catholic University   273159  District of Columbia    DC   
1  Washington            McLean Gardens   121759  District of Columbia    DC   
2  Washington             Benning Ridge   403493  District of Columbia    DC   
3  Washington  Southwest Federal Center   403489  District of Columbia    DC   
4  Washington          Civic Betterment   403495  District of Columbia    DC   

                                            geometry  active_rest  \
0  POLYGON ((-77.00433 38.94064, -77.00423 38.940...         14.0   
1  POLYGON ((-77.07520 38.93977, -77.07475 38.938...          NaN   
2  POLYGON ((-76.93493 38.87296, -76.93501 38.872...          1.0   
3  POLYGON ((-77.01609 38.88757, -77.01552 38.887...         34.0   
4  POLYGON ((-76.93053 38.88062, -76.93040 38.880...          0.0   

   close_18_rest  close_19_rest  active_deli  close_18_deli  close_19_deli  \
0            1.0            1.0          7.0            1.0            2.0   
1            NaN            NaN          NaN            NaN            NaN   
2            1.0            0.0          0.0            0.0            1.0   
3            2.0            1.0         11.0            0.0            3.0   
4            0.0            0.0          3.0            2.0            1.0   

   active_tot  close_18_tot  close_19_tot  pct_closed_rest  pct_closed_deli  \
0        21.0           2.0           3.0             12.5             30.0   
1         NaN           NaN           NaN              NaN              NaN   
2         1.0           1.0           1.0              0.0              0.0   
3        45.0           2.0           4.0              8.1             21.4   
4         3.0           2.0           1.0              0.0              0.0   

   pct_closed_tot     sq_mi  
0            19.2  2.263850  
1             NaN  0.190896  
2             0.0  0.837476  
3            11.8  0.743870  
4             0.0  0.117713

In [12]:
cols = dc_gdf.columns[6:17]

dc_gdf[cols] = dc_gdf[cols].fillna(0)

In [13]:
## create function that calculate the number of establishments per square mile
def sq_mi(col1,col2):
    if col1 == 0:
        return 0
    else:
        return round(col1/col2,0)
    
dc_gdf['active_tot_sq_mi'] = dc_gdf.apply(lambda x: sq_mi(x['active_tot'],x['sq_mi']), axis = 1)      
dc_gdf['active_rest_sq_mi'] = dc_gdf.apply(lambda x: sq_mi(x['active_rest'],x['sq_mi']), axis=1)      
dc_gdf['active_deli_sq_mi'] = dc_gdf.apply(lambda x: sq_mi(x['active_deli'],x['sq_mi']), axis=1)      

dc_gdf.head()

city                      name regionid                county state  \
0  Washington       Catholic University   273159  District of Columbia    DC   
1  Washington            McLean Gardens   121759  District of Columbia    DC   
2  Washington             Benning Ridge   403493  District of Columbia    DC   
3  Washington  Southwest Federal Center   403489  District of Columbia    DC   
4  Washington          Civic Betterment   403495  District of Columbia    DC   

                                            geometry  active_rest  \
0  POLYGON ((-77.00433 38.94064, -77.00423 38.940...         14.0   
1  POLYGON ((-77.07520 38.93977, -77.07475 38.938...          0.0   
2  POLYGON ((-76.93493 38.87296, -76.93501 38.872...          1.0   
3  POLYGON ((-77.01609 38.88757, -77.01552 38.887...         34.0   
4  POLYGON ((-76.93053 38.88062, -76.93040 38.880...          0.0   

   close_18_rest  close_19_rest  active_deli  ...  active_tot  close_18_tot  \
0            1.0            1.0          7.0  ...        21.0           2.0   
1            0.0            0.0          0.0  ...         0.0           0.0   
2            1.0            0.0          0.0  ...         1.0           1.0   
3            2.0            1.0         11.0  ...        45.0           2.0   
4            0.0            0.0          3.0  ...         3.0           2.0   

   close_19_tot  pct_closed_rest  pct_closed_deli  pct_closed_tot     sq_mi  \
0           3.0             12.5             30.0            19.2  2.263850   
1           0.0              0.0              0.0             NaN  0.190896   
2           1.0              0.0              0.0             0.0  0.837476   
3           4.0              8.1             21.4            11.8  0.743870   
4           1.0              0.0              0.0             0.0  0.117713   

   active_tot_sq_mi  active_rest_sq_mi  active_deli_sq_mi  
0               9.0                6.0                3.0  
1               0.0                0.0                0.0  
2               1.0                1.0                0.0  
3              60.0               46.0               15.0  
4              25.0                0.0               25.0  

[5 rows x 22 columns]

In [14]:
print(dc_gdf.shape)

(137, 22)
